In [205]:
import pandas as pd
import numpy as np
import re   
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import string
import collections
import nltk  
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk.corpus import stopwords, wordnet
from nltk import PorterStemmer, WordNetLemmatizer  
import string
from decimal import *
import sys
from nltk.stem import PorterStemmer
from nltk import ne_chunk, pos_tag, word_tokenize
import collections
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim import corpora, models, similarities
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel  
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 500)

[nltk_data] Downloading package punkt to /Users/jingyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jingyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jingyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jingyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/jingyan/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/jingyan/nltk_data...
[nltk_data]   Package words is already up-to-date!


### 1. Import and read-in  

In [206]:
data=pd.read_csv('takehome_ml_data.csv')
data.head()

,Unnamed: 0,question,question_id,question_type,survey_id
0,0,"In this survey, you will be presented with two versions of ""Getting Started"" options within a mortgage section of a bank's website. ​​​​​​​Please take a few minutes to answer the following questions. Remember to keep the link to the first design version open in a separate window so you can later compare it with the second design version that will be presented to you later on. Thank you in advance for your participation.",0,description,216
1,1,Which of the below best describes your current status?,2,single-select,216
2,2,"Referring to the image you have open... ​​​​​​​ If you are purchasing your first home and are interested in using a bank to get a mortgage, which option (if any) would you choose ?",3,single-select,216
3,3,What are your impressions of the options you were provided with?,4,rate,216
4,4,How clear or unclear are the options you were provided with?,5,single-select,216


In [207]:
data['question_type'].value_counts() #8 question types; 

single-select    607
textarea         541
rate             285
description      271
multi-select     126
matrix-likert    106
number           10 
rank             6  
Name: question_type, dtype: int64

### 2. Cleaning question texts   

In [208]:
data['question_c'] = data['question'].fillna('')
data['question_c']=data['question_c'].str.lower()
data['question_c']=data['question_c'].str.split()
data['question_c']=data['question_c'].apply(lambda x: [re.sub('[^a-zA-Z]*', '', i) for i in x])
data['question_c']=data['question_c'].apply(lambda x: [c for c in x if not c.isdigit()])

'''
def get_wordnet_pos(tag):  
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN 
    elif tag.startswith('R'):  
        return wordnet.ADV
    else:
        return wordnet.NOUN

def normalize(text):
    word_pos = nltk.pos_tag(nltk.word_tokenize(text))
    lemm_words = [nltk.WordNetLemmatizer().lemmatize(w[0], get_wordnet_pos(w[1])) for w in word_pos]  
    return [x.lower() for x in lemm_words]
data['question_c']=data['question_c'].apply(lambda x: ' '.join(x))
data['question_c']=data['question_c'].apply(normalize)
'''

a={'what', 'which', 'who', 'whom', 'or', 'above', 'below', 'when', 'where', 'why', 'how', 'all', 'any', 
'both', 'each', 'more', 'most', 'no', 'nor', 'not', 'only', 'than', 'very'} 
b={'would'}         
c=set(stopwords.words('english'))
sw=c|b 
sw=sw-a
data['question_c']=data['question_c'].apply(lambda x: ' '.join([c for c in x if c not in sw]))

def dfdic(text):    
    dic={}  
    for i in text: 
        for j in set(i.split()):
            if j in dic:
                dic[j]+=1
            else:    
                dic[j]=1  
    return dic  
dic=dfdic(data['question_c'])  
sw2={k for k, v in dic.items() if len(k)==1 or v==1}  
data['question_c']=data['question_c'].apply(lambda x: ' '.join([c for c in x.split() if c not in sw2]))

##### Lemmatization/stemming is opted out above since plural format of words (esp. nouns) helps to differentiate some question types (e.g. single-select v.s. multi-select). Omitting it turned out better predicition performance.    

### 3. Top 10 most frequently appeared words under each questioin type 

In [209]:
rank=10  #top 10 words;     
res={}
for c in data['question_type'].unique():
    dic=dfdic(data.loc[data['question_type']==c, 'question_c'])
    line=sorted(dic.values(), reverse=True)[rank:(rank+1)][0]   
    keywrd=[k for k, v in dic.items() if v>line]
    res[c]=keywrd    
res

{'description': ['link',
  'survey',
  'please',
  'take',
  'click',
  'below',
  'next',
  'page',
  'feedback',
  'like'],
 'single-select': ['which',
  'best',
  'how',
  'or',
  'following',
  'what',
  'use',
  'product',
  'find'],
 'rate': ['what',
  'how',
  'likely',
  'rate',
  'reaction',
  'or',
  'product',
  'find',
  'easy',
  'page'],
 'multi-select': ['which',
  'please',
  'select',
  'following',
  'or',
  'what',
  'how',
  'use',
  'any'],
 'textarea': ['why',
  'please',
  'what',
  'how',
  'like',
  'most',
  'make',
  'describe',
  'or',
  'product'],
 'matrix-likert': ['terms',
  'how',
  'following',
  'rate',
  'not',
  'or',
  'important',
  'satisfied',
  'current',
  'na'],
 'number': ['product',
  'price',
  'what',
  'think',
  'begin',
  'expensive',
  'consider'],
 'rank': ['order',
  'least',
  'first',
  'please',
  'following',
  'where',
  'most',
  'last',
  'rank']}

#### As seen here, question types do share similarities. For example, words such as ['which', 'following', 'what', 'or'], appear most often for both multi-select and single-select types. Words ['how', 'or', 'find] show up as top 10 in both rate and single-select question types. This would cause confusion in prediction later, and looking at the samples, even human being sometimes cannot differentiate. 

### 4. Feature engineering to boost prediction performance 

In [210]:
data=data.loc[:, ['question_c', 'question_type', 'question']]
data['any_fol']=data['question_c'].str.contains('any') & data['question_c'].str.contains('following')  
data['all_app']=data['question_c'].str.contains('all') & (data['question_c'].str.contains('apply') | data['question_c'].str.contains('select')) 

'''
from textblob import TextBlob
def sent(text):  
    sent=TextBlob(text).sentiment  
    return (sent[0], sent[1])  

data['pol']=data['question'].apply(lambda x: sent(x)[0])
data['sub']=data['question'].apply(lambda x: sent(x)[1])
'''

"\nfrom textblob import TextBlob\ndef sent(text):  \n    sent=TextBlob(text).sentiment  \n    return (sent[0], sent[1])  \n\ndata['pol']=data['question'].apply(lambda x: sent(x)[0])\ndata['sub']=data['question'].apply(lambda x: sent(x)[1])\n"

#### The sentiment such as polarity and subjectivity were also created (either based on original or cleaned question descriptions), but none turned out to improve model's prediction in a stable way. Code blocks thus commented out. 

### 5. Classification modeling 

In [211]:
Y=data[['question_type', 'any_fol', 'all_app']]  
X=data['question_c']

rs=184  
x_train, x_test, z_train, z_test = train_test_split(X, Y, test_size=0.25, random_state=rs)     
ytrain=z_train['question_type']  
ytest=z_test['question_type']

tfidfv=TfidfVectorizer(min_df=1, ngram_range=(1, 2), stop_words=None, max_features=10000, 
                       strip_accents='unicode', norm='l2')    
xtrain=tfidfv.fit_transform(x_train).todense()
xtest=tfidfv.transform(x_test).todense()  

col = ['feat_'+i for i in tfidfv.get_feature_names()]     
xtrain = pd.DataFrame(xtrain, columns=col)    
xtest = pd.DataFrame(xtest, columns=col) 

ztrain=pd.DataFrame(z_train.drop('question_type', axis=1)).reset_index(drop=True)
ztest=pd.DataFrame(z_test.drop('question_type', axis=1)).reset_index(drop=True)

xtrain=pd.concat([xtrain, ztrain], axis=1)
xtest=pd.concat([xtest, ztest], axis=1)

In [212]:
from sklearn.linear_model import LogisticRegression  
from sklearn.ensemble import RandomForestClassifier

rs=95
model=RandomForestClassifier(random_state=rs, n_estimators=80)
cname='RandomForest'  

model.fit(xtrain, ytrain)
pred=model.predict(xtest)
predproba=pd.DataFrame(model.predict_proba(xtest))
      
print(cname + ' predictions of test data'+'\n')  
print('Accuracy')
print(accuracy_score(ytest, pred, normalize=True))

RandomForest predictions of test data

Accuracy
0.8504098360655737


In [213]:
print('Confusion matrix')
pd.crosstab(ytest, pred, rownames=['Ground'], colnames=['Predicted'], margins=True)      

Confusion matrix


Predicted,description,matrix-likert,multi-select,number,rate,single-select,textarea,All
Ground,,,,,,,,
description,62,0,0,0,0,4,0,66
matrix-likert,0,19,0,0,2,0,0,21
multi-select,0,0,14,0,0,8,8,30
number,0,0,0,4,0,0,0,4
rate,0,0,0,0,54,10,2,66
single-select,3,0,5,0,7,144,12,171
textarea,0,0,0,0,1,11,118,130
All,65,19,19,4,64,177,140,488


In [214]:
print('Classification Report')    
report={}    
lines=classification_report(ytest, pred).split('\n')    
for l in lines[2:-5]: 
    report[l.split()[0]]=l.split()[1:]  
reportdf=pd.DataFrame.from_dict(report, orient='index', columns=['precision', 'recall', 'f1-score', 'num_samples'])    
reportdf  

Classification Report


,precision,recall,f1-score,num_samples
description,0.95,0.94,0.95,66
matrix-likert,1.00,0.90,0.95,21
multi-select,0.74,0.47,0.57,30
number,1.00,1.00,1.00,4
rate,0.84,0.82,0.83,66
single-select,0.81,0.84,0.83,171
textarea,0.84,0.91,0.87,130


In [216]:
print('Check on misclassification samples:')   
a=pd.DataFrame(x_test)
b=pd.DataFrame(ytest)
res_pred=pd.concat([a, b], axis=1)  
res_pred['pred']=pred    
res_pred['predprob']=[i.max() for i in model.predict_proba(xtest)]  
print('   Ground truth "multi-select" V.S. predicted "single-select"')
res_pred[(res_pred['question_type']=='multi-select') & (res_pred['pred']=='single-select')].sort_index()

Check on misclassification samples:
   Ground truth "multi-select" V.S. predicted "single-select"


,question_c,question_type,pred,predprob
199,which following expect receive website signing registration form,multi-select,single-select,0.5125
537,indicated organization wifi organization also outsource following,multi-select,single-select,0.5125
635,imagine service offered prescription refills delivery preferred method service,multi-select,single-select,0.4750
1037,which following features product like willing pay more access,multi-select,single-select,0.5500
1201,how research vehicle purchasing or leasing,multi-select,single-select,0.5625
1307,what level education child or children,multi-select,single-select,0.6500
1345,which products use,multi-select,single-select,0.6625
1743,teach course any following areas,multi-select,single-select,0.4750


#### Notes and conclusions:   
(1) The major stumbling block to the performance is in confusion between multi-select (ground trugh) and single-select/textareas (predict-to-be).           
(2) Checking both the underlying data (final step) and words with top frequency, it seems that human being may also unable to make good judgements (on the mis-classified cases), simply based off the question descriptions. It may be helpful for survey makers to add sentence e.g. "click all that apply" to help survey audience.                                                                                                                                             (3) Different data samplings have been tested, the model displayed good accuracies quite stably.        
(4) Also tested boosting models, it didn't outperform RandomForest to significant degree in this case.